# Import Libraries

In [ ]:
import os
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, \
    confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import ResNet50, DenseNet121
from tensorflow.keras.layers import Input, InputLayer, Conv2D, MaxPool2D, GlobalAveragePooling2D, BatchNormalization, Activation, ReLU, Flatten, Dense, Add,\
    Dropout,MaxPooling2D,Concatenate,Reshape
from tensorflow.keras.models import Sequential, Model
import tensorflow as tf
import warnings
from pathlib import Path
# Tüm uyarıları kapat
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt
import seaborn as sns

# Create the labels and enter the Data_Path.

In [ ]:
DATA_PATH = Path("/kaggle/input/potato-leaf-disease-dataset/Potato Leaf DIsease")
labels = ['Early Blight', 'Fungal Diseases','Healthy','Late Blight','Plant Pests','Potato Cyst Nematode','Potato Virus']

# Setting Seeds

In [ ]:
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Since we don't have a CSV file, we are creating a CSV file that contains our images including File, Disease Id, and Disease Type.

In [ ]:
data=[]
for disease_id , sp in enumerate(labels):
    for file in os.listdir(os.path.join(DATA_PATH, sp)):
        data.append(['{}/{}'.format(sp, file), disease_id, sp])
csv_data = pd.DataFrame(data, columns=['File', 'Disease Id', 'Disease Type'])
csv_data

# Check if there is a null value in data.

In [ ]:
csv_data.isnull().sum()

# Creating our image reading and resizing functions.

In [ ]:
IMAGE_SIZE = 112

def read_image(filepath):
    return cv2.imread(os.path.join(DATA_PATH, filepath))
def resize_image(image, image_size):
    return cv2.resize(image.copy(), image_size, interpolation=cv2.INTER_AREA)

# Read Images

In [ ]:
X_data_item = np.zeros((csv_data.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3))
for i, file in tqdm(enumerate(csv_data['File'].values)):
    image = read_image(file)
    if image is not None:
        X_data_item[i] = resize_image(image, (IMAGE_SIZE, IMAGE_SIZE))
# Normalize data
X_data = X_data_item #/ 255.0
print('Train Shape: {}'.format(X_data.shape))

Y_data = csv_data['Disease Id'].values
Y_data = to_categorical(Y_data)

# Look at how many there are in each label.

In [ ]:
class_counts = csv_data['Disease Type'].value_counts()
plt.figure(figsize=(8, 4))
sns.barplot(x=class_counts.index, y=class_counts.values)
plt.xlabel('Labels')
plt.ylabel('Number of Samples')
plt.title('Label Distribution in the Dataset')
plt.xticks(rotation=45)
plt.show()

# Visualize 5 images randomly

In [ ]:
from PIL import Image

_, axs = plt.subplots(1, 5, figsize=(20, 5))

for j in range(5):
    index = np.random.randint(0,len(csv_data))
    img_path = csv_data.iloc[index]['File']
    label = csv_data.iloc[index]['Disease Type']
    img = Image.open(str(DATA_PATH)+"/"+img_path)
    axs[j].imshow(img)
    axs[j].set_title(label)
plt.tight_layout()
plt.show()

# Split images data to X_train, Y_train, X_val, Y_val, X_test, Y_test

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_data, Y_data, test_size=0.2)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2)

In [ ]:
X_train.shape, Y_train.shape, X_val.shape, Y_val.shape, X_test.shape, Y_test.shape

# Create the Inception model and add layers to make it suitable for dataset.

In [ ]:
basemodel = InceptionV3(input_shape=(IMAGE_SIZE,IMAGE_SIZE,3), include_top=False, weights='imagenet', classes=7)

flatten_layer = Flatten()(basemodel.output)
dense_layer = Dense(32, activation='relu')(flatten_layer)
# Dropout ve çıkış katmanı
dropout_layer = Dropout(0.4)(dense_layer)
output_layer = Dense(7, activation='softmax')(dropout_layer)

model = Model(inputs=basemodel.input, outputs=output_layer);

# Create adam optimizer and compile the model

In [ ]:
from keras.metrics import Precision, Recall,AUC,F1Score
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # lr=0.0001 , 0.1
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', Precision(), Recall(), AUC(), F1Score()])
# model.summary() ön eğitimli ağda çok fazla katman bulunduğu için boş yer kaplıyor

# Create callbacks

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau

# Kendi Callbackim
class EarlyStoppingAtThreshold(tf.keras.callbacks.Callback):# kerasın içindeki callback den kalıtım aldık
    def __init__(self, monitor, threshold, patience = 0, restore_best_weights=False):
        super(EarlyStoppingAtThreshold, self).__init__()
        self.monitor = monitor # izlenmesi gereken metrik
        self.threshold = threshold # hangi değerde duracağımız
        self.patience = patience # o değere ulaştıktan sonra kaç epoch devam etsin
        self.restore_best_weights = restore_best_weights # en iyi ağırlıklar
        self.wait = 0 # bunu wait > patience için tanımlandı
        self.best_weights = None # en iyi ağırlıklarımızı tutacak
        self.best = -float('inf') # metriğin en yüksek değeri

    def on_train_begin(self, logs=None): # eğitime başlangıç anında olacaklar
        self.wait = 0
        self.best = -float('inf')
        if self.restore_best_weights: # True ise modelin ilk ağırlığını best_weights e atacak
            self.best_weights = self.model.get_weights()

    def on_epoch_end(self, epoch, logs=None): # Her epoch sonunda olacaklar
        current = logs.get(self.monitor) # izlencek metriğin değerini çekiyoruz
        if current is None:
            print(f"Warning: Metric '{self.monitor}' is not available.")
            return

        if current > self.best: # eğer metriğin değeri öncekinden yüksekse
            self.best = current # en yüksek metirk değerini değiştiriyoruz
            self.wait = 0
            if self.restore_best_weights: # true ise
                self.best_weights = self.model.get_weights() # en iyi ağırlığı güncelliyoruz
        elif current >= self.threshold: # metriğin değeri istediğimiz değeri geçtiyse
            self.wait += 1 # beklemeyi 1 arttırıyoruz
            if current > self.best:
                self.best = current
                if self.restore_best_weights: # true ise
                    self.best_weights = self.model.get_weights()
            if self.wait > self.patience: # wait > patience olunca
                # istediğimiz değere geldi durduruyoruz
                print(f"\nReached {self.threshold*100}% {self.monitor}. Stopping training after {self.patience} more epochs.")
                if self.restore_best_weights:# True ise
                    print("Restoring model weights from the best epoch.")
                    self.model.set_weights(self.best_weights)# en iyi ağırlıkları ayarlıyoruz
                self.model.stop_training = True #modeli durduruyoruz

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.4, patience=5, cooldown=3, min_lr=1e-4, verbose=1)

early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=10, restore_best_weights=False)

early_stoppin_at_threshold = EarlyStoppingAtThreshold(monitor='val_accuracy', threshold=0.94, patience = 0, restore_best_weights=True)

calbacks=[early_stopping,reduce_lr,early_stoppin_at_threshold]

# Train Model

In [ ]:
batch_size = 64
epochs = 50
hist_concat = model.fit(
    [X_train],Y_train,
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True,
    validation_data=([X_val],Y_val),
    callbacks=calbacks
    )

# Test Model

In [ ]:
y_pred = model.predict([X_test])
y_pred = np.argmax(y_pred, axis=1).reshape(-1, 1)
Y_test = np.argmax(Y_test, axis=1).reshape(-1, 1)

# Model Results and Charts

In [ ]:
def model_result(Y_test,y_pred):
    print(" MODEL RESULTS")
    print("Accuracy: ", accuracy_score(Y_test, y_pred))
    print("F1_Score: ", f1_score(Y_test, y_pred, average='macro'))
    print("Precision: ", precision_score(Y_test, y_pred, average='macro'))
    print("Sensitivity: ", recall_score(Y_test, y_pred, average='macro'))

In [ ]:
import matplotlib.pyplot as plt
def Plot_acc_history(hist_concat):
    plt.plot(hist_concat.history['accuracy'])
    plt.plot(hist_concat.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
def Plot_loss_history(hist_concat):
    plt.plot(hist_concat.history['loss'])
    plt.plot(hist_concat.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix
def Confusion_matrix(Y_test,y_pred):
    class_names = labels
    print('Test Confusion Matrix')
    cm_dense = confusion_matrix(Y_test, y_pred)
    sn.set(font_scale=1.2)  # for label size
    sn.heatmap(cm_dense, annot=True, fmt="d", linewidths=.5, annot_kws={"size": 16},xticklabels=class_names, yticklabels=class_names)

In [ ]:
model_result(Y_test,y_pred)

In [ ]:
Plot_acc_history(hist_concat)
Plot_loss_history(hist_concat)

In [ ]:
Confusion_matrix(Y_test,y_pred)

# Create the Resnet50 model and add layers to make it suitable for dataset.

In [ ]:
basemodel = ResNet50(input_shape=(IMAGE_SIZE,IMAGE_SIZE,3), include_top=False, weights='imagenet', classes=7)

flatten_layer = Flatten()(basemodel.output)
dense_layer = Dense(32, activation='relu')(flatten_layer)
# Dropout ve çıkış katmanı
dropout_layer = Dropout(0.4)(dense_layer)
output_layer = Dense(7, activation='softmax')(dropout_layer)

model = Model(inputs=[basemodel.input], outputs=output_layer);

In [ ]:
from keras.metrics import Precision, Recall,AUC,F1Score
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # lr=0.0001 , 0.1
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', Precision(), Recall(), AUC(), F1Score()])
# model.summary() ön eğitimli ağda çok fazla katman bulunduğu için boş yer kaplıyor

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.4, patience=5, cooldown=3, min_lr=1e-4, verbose=1)

early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=10, restore_best_weights=True)

#early_stoppin_at_threshold = EarlyStoppingAtThreshold(monitor='val_accuracy', threshold=0.93, patience = 2, restore_best_weights=True)

calbacks=[early_stopping,reduce_lr]

In [ ]:
batch_size = 64
epochs = 50
hist_concat = model.fit(
    [X_train],Y_train,
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True,
    validation_data=([X_val],Y_val),
    callbacks=calbacks
    )

In [ ]:
y_pred = model.predict([X_test])
y_pred = np.argmax(y_pred, axis=1).reshape(-1, 1)
Y_test = np.argmax(Y_test, axis=1).reshape(-1, 1)

In [ ]:
model_result(Y_test,y_pred)

In [ ]:
Plot_acc_history(hist_concat)
Plot_loss_history(hist_concat)

In [ ]:
Confusion_matrix(Y_test,y_pred)

In [ ]:
basemodel = InceptionV3(input_shape=(IMAGE_SIZE,IMAGE_SIZE,3), include_top=False, weights='imagenet', classes=7)
basemodel2 = ResNet50(include_top=False)

for layer in basemodel.layers:
    layer.trainable = True
for layer in basemodel2.layers:
    layer.trainable = True

global_pooling_layer1 = GlobalAveragePooling2D()(basemodel.output)
global_pooling_layer2 = GlobalAveragePooling2D()(basemodel2.output)

global_pooling_layer1 = Reshape((1, 1, global_pooling_layer1.shape[1]))(global_pooling_layer1)
global_pooling_layer2 = Reshape((1, 1, global_pooling_layer2.shape[1]))(global_pooling_layer2)

concatenated_output = Concatenate()([global_pooling_layer1, global_pooling_layer2])

flatten_layer = Flatten()(concatenated_output)

dense_layer = Dense(32, activation='relu')(flatten_layer)
# Dropout ve çıkış katmanı
dropout_layer = Dropout(0.4)(dense_layer)
output_layer = Dense(7, activation='softmax')(dropout_layer)

model = Model(inputs=[basemodel.input,basemodel2.input], outputs=output_layer)

In [ ]:
from keras.metrics import Precision, Recall,AUC,F1Score
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # lr=0.0001 , 0.1
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', Precision(), Recall(), AUC(), F1Score()])
# model.summary() ön eğitimli ağda çok fazla katman bulunduğu için boş yer kaplıyor

In [ ]:
batch_size = 64
epochs = 50
hist_concat = model.fit(
    [X_train,X_train],Y_train,
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True,
    validation_data=([X_val,X_val],Y_val),
    callbacks=calbacks
    )

In [ ]:
y_pred = model.predict([X_test,X_test])
y_pred = np.argmax(y_pred, axis=1).reshape(-1, 1)
Y_test = np.argmax(Y_test, axis=1).reshape(-1, 1)

In [ ]:
model_result(Y_test,y_pred)

In [ ]:
Plot_acc_history(hist_concat)
Plot_loss_history(hist_concat)

In [ ]:
Confusion_matrix(Y_test,y_pred)

# Create My CNN


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout

model=Sequential()
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dense(7, activation='softmax'))

In [ ]:
from keras.metrics import Precision, Recall,AUC,F1Score
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # lr=0.0001 , 0.1
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', Precision(), Recall(), AUC(), F1Score()])
model.summary()

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.4, patience=5, cooldown=3, min_lr=1e-4, verbose=1)

early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=10, restore_best_weights=True)

#early_stoppin_at_threshold = EarlyStoppingAtThreshold(monitor='val_accuracy', threshold=0.93, patience = 2, restore_best_weights=True)

calbacks=[early_stopping,reduce_lr]

In [ ]:
batch_size = 64
epochs = 50
hist_concat = model.fit(
    [X_train],Y_train,
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True,
    validation_data=([X_val],Y_val),
    callbacks=calbacks
    )

In [ ]:
y_pred = model.predict([X_test])
y_pred = np.argmax(y_pred, axis=1).reshape(-1, 1)
Y_test = np.argmax(Y_test, axis=1).reshape(-1, 1)

In [ ]:
model_result(Y_test,y_pred)

In [ ]:
Plot_acc_history(hist_concat)
Plot_loss_history(hist_concat)

In [ ]:
Confusion_matrix(Y_test,y_pred)